In [4]:
%load_ext autoreload
%autoreload 2

import torch
import pyro
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bb_opt.src.bayesian_opt import (
    optimize,
    get_model_bnn,
    train_model_bnn,
    partial_train_model_bnn,
)
from bb_opt.src.utils import get_path
from gpu_utils.utils import gpu_init

gpu_id = gpu_init()
print(f"Running on GPU {gpu_id}")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Running on GPU 0


In [3]:
def jointplot(predicted, true, title: str=""):
    if isinstance(predicted, torch.Tensor):
        predicted = predicted.detach().cpu().numpy()

    if isinstance(true, torch.Tensor):
        true = true.detach().cpu().numpy()
        
    ax = sns.jointplot(predicted, true, s=3, alpha=0.5)
    ax.set_axis_labels('Predicted', 'True')
    ax.ax_marg_x.set_title(title)
    return ax

In [5]:
project = "dna_binding"
dataset = "crx_ref_r1"
batch_size = 1
retrain_every = 1000000
partial_steps = 20

root = "/cluster/nhunt/github/bb_opt/"
data_dir = get_path(root, "data", project, dataset)
inputs = np.load(get_path(data_dir, "inputs.npy"))
labels = np.load(get_path(data_dir, "labels.npy"))

top_k_percent = 1

In [8]:
bnn_model, guide = get_model_bnn(n_inputs=inputs.shape[1])
optimizer = pyro.optim.Adam({})

model_name = "../models/crx_1000"
pyro.get_param_store().load(f"{model_name}.params")
optimizer.load(f"{model_name}.opt")

inputs = torch.tensor(inputs, dtype=torch.float32, device=device)
labels = torch.tensor(labels, dtype=torch.float32, device=device)